### Analysis2.ipynb
Clustering SBERT embeddings using UMAP and Fuzzy C-Means.

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import skfuzzy
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import umap

In [ ]:
# Set paths
root = os.getcwd()
input_path = os.path.join(root, 'data', 'SBERT', 'SBERT-L6-v2_embed.csv')
output_path = os.path.join(root, 'output')
os.makedirs(output_path, exist_ok=True)

In [ ]:
# Load SBERT embeddings
data = pd.read_csv(input_path)
embedding = data.values

In [ ]:
# Dimensionality reduction using UMAP
umap_model = umap.UMAP(n_neighbors=30, min_dist=0.1, n_components=2, random_state=42)
umap_result = umap_model.fit_transform(embedding)

In [ ]:
# Fuzzy C-Means clustering
fuzz_data = umap_result.T
n_clusters = 4
cntr, u, _, _, _, _, _ = skfuzzy.cluster.cmeans(
    fuzz_data, c=n_clusters, m=2, error=0.005, maxiter=1000, init=None)

In [ ]:
# Assign dominant cluster and save results
fuzz_membership = u.T
dominant_cluster = np.argmax(fuzz_membership, axis=1)
clustered_data = pd.DataFrame(umap_result, columns=['Feature_1', 'Feature_2'])
clustered_data['fuzzy_cluster'] = dominant_cluster
for i in range(n_clusters):
    clustered_data[f'fuzzy_membership_{i}'] = fuzz_membership[:, i]
clustered_data.to_csv(os.path.join(output_path, 'MEAM_fuzzy_clustered.csv'), index=False)